In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn import preprocessing

#model building
import torch
import torchvision
from torchvision.transforms import v2
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from time import sleep


#plotting and evalueation
import seaborn as sns


DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
#reading the data
kenya_train_df = pd.read_csv("geoai-challenge-for-agricultural-plastic-cover-mapping-with-satellite-imagery20240708-24674-1c1nnx3/Kenya_training.csv")
kenya_test_df = pd.read_csv("geoai-challenge-for-agricultural-plastic-cover-mapping-with-satellite-imagery20240708-24674-1c1nnx3/Kenya_training.csv")
kenya_test_df.head(5)

,ID,lon,lat,blue_p50,green_p50,nir_p50,nira_p50,re1_p50,re2_p50,re3_p50,red_p50,swir1_p50,swir2_p50,VV_p50,VH_p50,TARGET
0,1,37.059054,0.046039,2731.0,3035.0,5418.0,5306.5,2958.0,4610.5,5099.5,2994.0,4108.5,3187.5,-6.861458,-12.162713,1
1,2,37.059503,0.047296,2921.0,3329.0,5834.0,6472.0,4144.5,5782.0,6302.0,3344.0,5269.0,4471.5,-8.082717,-14.619608,1
2,3,37.057527,0.047656,3828.0,4163.0,6420.0,7890.5,5408.5,6695.5,7302.0,4380.0,6905.0,6905.5,-7.767043,-14.880353,1
3,4,37.057976,0.047925,2556.0,2839.0,4822.0,5276.5,3201.0,4643.0,5069.5,2787.0,3832.0,2867.5,-7.773118,-15.587653,1
4,5,37.056359,0.048554,3790.0,4009.0,6476.0,6199.5,4285.0,5711.0,6079.5,3939.0,4892.5,3702.5,-6.104467,-15.431915,1


In [3]:
#Dropping the id Variable
y_train = (kenya_train_df["TARGET"]-1).to_numpy()
y_test = (kenya_test_df["TARGET"]-1).to_numpy()


#Dropping the id column
kenya_train_df = kenya_train_df.drop('ID', axis = 1)
kenya_test_df = kenya_test_df.drop('ID', axis = 1)

x_train = kenya_train_df.to_numpy()
x_test = kenya_test_df.to_numpy()


In [4]:
#normalizing the data 
x_train = preprocessing.normalize(x_train)
x_test = preprocessing.normalize(x_test)


#Turning the data into torch torch tensors for gpu acceleration
x_train = torch.from_numpy(x_train).type(torch.float)
x_test = torch.from_numpy(x_test).type(torch.float)
y_train = torch.from_numpy(y_train).type(torch.float)
y_test = torch.from_numpy(y_test).type(torch.float)


In [5]:
#Defining Custom Dataset
class KenyaDataset(Dataset):
    def __init__(self, x, y):
        self.features = x
        self.labels = y

    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self. labels[idx]


In [6]:
batch_size = 32

#creating custom Datasets
train_dataset = KenyaDataset(x_train, y_train)
test_dataset = KenyaDataset(x_test, y_test)

#creating custom DataLoaders
train_dataloader = DataLoader(train_dataset,batch_size = batch_size, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

In [7]:
#Creating the Module
class First_Model(nn.Module):
    def __init__(self):
        super(First_Model, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(15, 128),  # Input size 15 features
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 1),   # Output size is 1 for binary classification
            nn.Sigmoid()         # Use Sigmoid for binary classification
        )

    def forward(self, x):
        return self.net(x)


In [16]:
def training(model, loss_fn, train_loader, opt, epochs = 1000):


    running_losses = []
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0 
        for i, (datapoints, labels) in enumerate(tqdm(train_loader)):


            #forward proporgation
            outputs = model(datapoints)

            labels = labels.float().unsqueeze(1).float()
            loss = loss_fn(outputs, labels)

            #backpropogation
            opt.zero_grad()
            loss.backward()
            opt.step()


            running_losses.append(loss.item())
            epoch_loss+=loss.item()

            # if (i+1) %10 == 0:
            #     print(f"epoch number {epoch+1}, loss = {loss.item()}")
    print("Training Finished")
    return running_losses


In [12]:
def calculate_accuracy(model, test_loader):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for datapoints, labels in test_loader:
            outputs = model(datapoints)

            probabilities = torch.sigmoid(outputs)

            predicted = (probabilities > 0.5).float()

            labels = labels.unsqueeze(1).float()

    
            correct += (predicted == labels).sum().item()
            total += labels.size(0)


    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy}%")
    return accuracy

In [17]:
model = First_Model()
loss_function  = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

running_losses = training(model = model, loss_fn = loss_function, train_loader = train_dataloader, opt = optimizer)

accuracy =  calculate_accuracy(model, test_dataloader)

100%|██████████| 32/32 [00:00<00:00, 1879.35it/s]

Training Finished
Test Accuracy: 91.28%
